## Homework №2

In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
punct = set(punctuation)
from sklearn.metrics import classification_report

In [2]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [3]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [4]:
from difflib import get_close_matches

In [5]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import textdistance

In [6]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [7]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [8]:
def get_closest_match_vec(text, X, vec, TOPN=3):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    
    return [id2word[top] for top in topn]

In [9]:
w = 'опофеоз'

In [10]:
closest_set = get_closest_match_vec(w, X, vec)
closest_set

['апофеоз', 'апофеозом', 'апофеоза']

In [11]:
def get_closest_match_with_metric(text, lookup, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
        
    return similarities.most_common(1)[0]

In [12]:
get_closest_match_with_metric(w, closest_set, textdistance.levenshtein)

('апофеоз', 0.8571428571428572)

In [13]:
def get_closest_hybrid_match(word, X, vec):
    candidates = get_closest_match_vec(word, X, vec)
    #print('candidates: ', candidates)
    sims = Counter()
    closest = get_closest_match_with_metric(word, candidates)   
    return closest

In [14]:
get_closest_hybrid_match(w, X, vec)

('апофеоз', 0.8571428571428572)

In [15]:
all_sent = []
for i in range(0, len(bad)):
    pairs = align_words(true[i],bad[i])
    for j in pairs:
        all_sent.append(j)
len(all_sent)

10012

In [16]:
pairs = all_sent

In [17]:
clos_set = []
for i in range(0,len(pairs)):
    closest = get_closest_hybrid_match(pairs[i][1], X, vec)
    clos_set.append(closest)
    #print('closest to "'+pairs[i][1]+'" is "'+closest[0]+'"')

In [18]:
success = 0
for i in range(0,len(pairs)):
    #print('bad: ', pairs[i][1])
    #print('true: ', pairs[i][0])
    #print('closest: ', clos_set[i], '\n')
    if clos_set[i][0] == pairs[i][0]:
        success += 1
print(success/len(pairs))

0.7947463044346784


In [19]:
s_all = 0
for i in range(0,len(pairs)):
    s_all += clos_set[i][1]
av_len_all = s_all/len(pairs)
print(av_len_all)

0.9153556665771331


Попробуем улучшить. Сначала я возьму только одно предложение для проверки, и сравню полученные результаты.

Продублирую один кусочек кода, и далее добавлю измененный.

In [20]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [21]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec_2 = TfidfVectorizer(analyzer='char', ngram_range=(1,4))
X_2 = vec_2.fit_transform(vocab)

In [22]:
pairs_small = align_words(true[0],bad[0])
pairs_small

[('симпатичнейшее', 'симпатичнейшое'),
 ('шпионское', 'шпионское'),
 ('устройство', 'устройство'),
 ('такой', 'такой'),
 ('себе', 'себе'),
 ('гламурный', 'гламурный'),
 ('фотоаппарат', 'фотоаппарат'),
 ('девушки', 'девушки'),
 ('бонда', 'бонда'),
 ('миниатюрная', 'миниатюрная'),
 ('модель', 'модель'),
 ('камеры', 'камеры'),
 ('superheadz', 'superheadz'),
 ('clap', 'clap'),
 ('camera', 'camera')]

In [23]:
clos_set_1 = []
for i in range(0,len(pairs_small)):
    closest_1 = get_closest_hybrid_match(pairs_small[i][1], X, vec)
    clos_set_1.append(closest_1)
    print('closest to "'+pairs_small[i][1]+'" is "'+closest_1[0]+'"')

closest to "симпатичнейшое" is "пластичнейшими"
closest to "шпионское" is "шпионские"
closest to "устройство" is "устройство"
closest to "такой" is "такой"
closest to "себе" is "себе"
closest to "гламурный" is "гуманный"
closest to "фотоаппарат" is "фотоаппарат"
closest to "девушки" is "девушки"
closest to "бонда" is "банд"
closest to "миниатюрная" is "миниатюрная"
closest to "модель" is "модель"
closest to "камеры" is "камеры"
closest to "superheadz" is "herederas"
closest to "clap" is "place"
closest to "camera" is "america"


In [24]:
success_1 = 0
for i in range(0,len(pairs_small)):
    #print('bad: ', pairs_small[i][1])
    #print('true: ', pairs_small[i][0])
    #print('closest: ', clos_set_1[i], '\n')
    if clos_set_1[i][0] == pairs_small[i][0]:
        success_1 += 1
print(success_1/len(pairs_small))

0.5333333333333333


In [25]:
s_1 = 0
for i in range(0,len(pairs_small)):
    s_1 += clos_set_1[i][1]
av_len_1 = s_1/len(pairs_small)
print(av_len_1)

0.7762962962962962


Теперь второй вариант.

In [26]:
clos_set_2 = []
for i in range(0,len(pairs_small)):
    closest_2 = get_closest_hybrid_match(pairs_small[i][1], X_2, vec_2)
    clos_set_2.append(closest_2)
    print('closest to "'+pairs_small[i][1]+'" is "'+closest_2[0]+'"')

closest to "симпатичнейшое" is "симпатичный"
closest to "шпионское" is "шпионской"
closest to "устройство" is "устройство"
closest to "такой" is "такой"
closest to "себе" is "себе"
closest to "гламурный" is "гламур"
closest to "фотоаппарат" is "фотоаппарат"
closest to "девушки" is "девушки"
closest to "бонда" is "бонди"
closest to "миниатюрная" is "миниатюрная"
closest to "модель" is "модель"
closest to "камеры" is "камеры"
closest to "superheadz" is "superjet"
closest to "clap" is "ap"
closest to "camera" is "tamer"


In [27]:
success_2 = 0
for i in range(0,len(pairs_small)):
    #print('bad: ', pairs_small[i][1])
    #print('true: ', pairs_small[i][0])
    #print('closest: ', clos_set_2[i], '\n')
    if clos_set_2[i][0] == pairs_small[i][0]:
        success_2 += 1
print(success_2/len(pairs_small))

0.5333333333333333


In [28]:
s_2 = 0
for i in range(0,len(pairs_small)):
    s_2 += clos_set_2[i][1]
av_len_2 = s_2/len(pairs_small)
print(av_len_2)

0.8557671957671957


Несмотря на то, что количество ошибок не изменилось и они допущены в одних и тех же местах, видно, что во втором случае, подбираемые слова, ближе к исходным. Например, для "симпатичнейшое" - "симпатичный", а не "пластичнейшими".

Я пробовала сделать также для всех предложений, но код долго работал (слишком долго), что никак не удавалось получить результат. Так что, возможно, это не самый лучший спобоб добиться каких-то улучшений.

In [ ]:
clos_set_3 = []
for i in range(0,len(pairs)):
    closest_3 = get_closest_hybrid_match(pairs[i][1], X_2, vec_2)
    clos_set_3.append(closest_3)
    #print('closest to "'+pairs[i][1]+'" is "'+closest_3[0]+'"')

In [ ]:
success_3 = 0
for i in range(0,len(pairs)):
    #print('bad: ', pairs[i][1])
    #print('true: ', pairs[i][0])
    #print('closest: ', clos_set_3[i], '\n')
    if clos_set_3[i][0] == pairs[i][0]:
        success_3 += 1
print(success_3/len(pairs))

In [ ]:
s_all_2 = 0
for i in range(0,len(pairs)):
    s_all_2 += clos_set_3[i][1]
av_len_all_2 = s_all_2/len(pairs)
print(av_len_all_2)